In [ ]:
import time
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta
import requests
from requests.exceptions import ConnectionError
import json
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

#  Адрес службы отчетов для отправки запросов JSON (с учетом регистра)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'
# Токен пользователя
token = 'Ваш токен'
# Обязательный параметр, если запросы подаются от имени рекламного агентства
clientLogin = 'Ваш логин ЯндексДирект'

# --- Preparing the request ---
#  Creating HTTP request headers
headers = {
    # OAuth token. The word Bearer must be used
    "Authorization": "Bearer " + token,
    # Login of the advertising agency client
    "Client-Login": clientLogin,
    # Language for response messages
    "Accept-Language": "en",
    # Mode for report generation
    "processingMode": "auto",
    # Format for monetary values in the report
     "returnMoneyInMicros": "false",
    # Don't include the row with the report name and date range in the report
    "skipReportHeader": "true",
    # Don't include the row with column names in the report
    # "skipColumnHeader": "true",
    # Don't include the row with the number of statistics rows in the report
    "skipReportSummary": "true"
}

# Получаем текущую дату
today = datetime.today()
first_day_of_current_month = today.replace(day=1)
# Определяем последний день прошлого месяца
dateE = first_day_of_current_month - timedelta(days=1)
# Определяем первый день прошлого месяца
dateS = dateE.replace(day=1)

# Creating the request message body

body = {
    "params": {
        "SelectionCriteria": {
            "DateFrom": dateS.strftime('%Y-%m-%d'),
            "DateTo": dateE.strftime('%Y-%m-%d')
        },
           "FieldNames": [
            "Date",
            "CampaignName",
            "Impressions",
            "Clicks",
            "Conversions",
            "ConversionRate",
            "Profit",
            "Bounces",
            "Cost",
            "Revenue",
            "Sessions"
        ],
        "Goals":['Ваша цель'], # Выбор цели
        "AttributionModels":['LYDC'], # Выбор модели атрибуции
        "ReportName": u('YD'),
        "Page": { 
            "Limit": 4000000
        },
        "ReportType": "CUSTOM_REPORT",
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",
            "IncludeVAT": "NO",
            "IncludeDiscount": "NO"
        }
    }

body = json.dumps(body, indent=4)

while True:
        try:
            req = requests.post(ReportsURL, body, headers=headers)
            req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(u(body)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 200:
                print("Отчет создан успешно")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                #print("Содержание отчета: \n{}".format(u(req.text)))
                break
            elif req.status_code == 201:
                print("Отчет успешно поставлен в очередь в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format('60'))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(60)
            elif req.status_code == 202:
                print("Отчет формируется в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 502:
                print("Время формирования отчета превысило серверное ограничение.")
                print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                print("JSON-код запроса: {}".format(body))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            else:
                print("Произошла непредвиденная ошибка")
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(body))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break


        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            # В данном случае мы рекомендуем повторить запрос позднее
            print("Произошла ошибка соединения с сервером API")
            # Принудительный выход из цикла
            break

        # Если возникла какая-либо другая ошибка
        except:
            # В данном случае мы рекомендуем проанализировать действия приложения
            print("Произошла непредвиденная ошибка")
            # Принудительный выход из цикла
            break


# Блок обработки выгрузки из ЯндексДиректа для расчета основных метрик            
months = {'February':2,'December':12,'January':1,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11}
raw_text: list[str] = req.text.split('\n')[:-1]
ready_table = pd.DataFrame({'data':raw_text[1:]})['data'].astype(str).str.split('\t',expand=True)
ready_table.columns = raw_text[0].split('\t')
ready_table['Month'] = pd.to_datetime(ready_table['Date']).dt.strftime('%B %y')
ready_table['Impressions'] = ready_table['Impressions'].apply(lambda x:int(x))
ready_table['Clicks'] = ready_table['Clicks'].apply(lambda x:int(x))
ready_table['Cost'] = ready_table['Cost'].apply(lambda x:float(x))
ready_table['Revenue'] = ready_table['Revenue_3051955286_LYDCCD'].apply(lambda x:float(x.replace('--','0')))
ready_table['Bounces'] = ready_table['Bounces'].apply(lambda x:int(x))
ready_table['Sessions'] = ready_table['Sessions'].apply(lambda x:int(x))
ready_table['Conversions'] = ready_table['Conversions_3051955286_LYDCCD'].apply(lambda x:int(x.replace('--','0')))
ready_table1 = ready_table[['Month','Impressions','Clicks','Cost','Conversions','Revenue','Sessions','Bounces']].groupby('Month').sum()
ready_table1['CTR'] = round(ready_table1['Clicks']/ready_table1['Impressions']*100,2)
ready_table1['CPC'] = round(ready_table1['Cost']/ready_table1['Clicks'],2)
ready_table1['CPO'] = round(ready_table1['Cost']/ready_table1['Conversions'],0)
ready_table1['ДРР'] = round(ready_table1['Cost']/ready_table1['Revenue']*100,2)
ready_table1['CR'] = round(ready_table1['Conversions']/ready_table1['Clicks']*100,2)
ready_table1['%отказы'] = round(ready_table1['Bounces']/ready_table1['Sessions']*100,2)
ready_table1 = ready_table1.reset_index()
ready_table1['Year'] = ready_table1['Month'].apply(lambda x: x.split()[1])
ready_table1['Month_number'] = ready_table1['Month'].apply(lambda x: months[x.split()[0]])
ready_table1 = ready_table1[['Year','Month_number','Month','Impressions','Clicks','CTR','CPC','Cost','Conversions','CPO','CR','Revenue','ДРР','%отказы']]

# Подключение к Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# Ссылка на ваш файл json с учетными данными для API Google Sheets. Данные файл можно получить при создании нового проекта в Google Cloud. 
creds = ServiceAccountCredentials.from_json_keyfile_name('Ваш путь к файлу/gs_credentials.json', scope)
client = gspread.authorize(creds)

# Открываем Google Sheets таблицу по её имени. Важно: в настройках доступа вашей таблицы нужно добавить доступ вашему сервисному аккаунту. Сервисный аккаунт вы получете при создании проекта в Google Cloud. 
sheet_name = 'Название вашей таблицы'
sheet = client.open(sheet_name).sheet1

# Т.к. таблица обновляемая с накопительными данными, данные будут записываться после последней заполненной строки.
# Получаем текущее количество заполненных строк
current_rows = len(sheet.get_all_values())

# Определяем диапазон ячеек для обновления, начиная с последней строки
start_row = current_rows + 1
end_row = start_row + len(ready_table1) - 1
update_range = f'A{start_row}:O{end_row}'

# Преобразуем DataFrame в список списков для обновления
update_values = ready_table1.values.tolist()

# Обновляем Google Sheets, добавляя новые значения, начиная с последней строки
sheet.update(update_range, update_values)

print("Данные успешно загружены в Google Sheets.")